In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Training import GetPilotNetDataset
from Training import GetDatasets
from Nets_original  import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from datetime import datetime
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import itertools
import pathlib
import os, sys
from openpyxl import Workbook
from openpyxl import load_workbook
from Simulation import buffer_simulation, save_obj, load_obj


tf.random.set_seed(1234)
np.random.seed(1234)

# 1) Training

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Get Dataset

In [2]:
trainBatchSize = testBatchSize = 32
train_set,valid_set,test_dataset = GetPilotNetDataset(csv_dir='Data/Datos/driving_log.csv', train_batch_size=32, test_batch_size=32)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b) Get Model

In [22]:
PilotNet = GetNeuralNetworkModel('PilotNet',(160,320,3),1,quantization = False,aging_active = False)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
# Compile Model
PilotNet.compile(optimizer=optimizer, loss='mse')

dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c) Training

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; d) Save/Load Weigths

In [23]:
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'PilotNet')
wgt_dir = os.path.join(wgt_dir,'Weights')
PilotNet.load_weights(wgt_dir)

# 2) Stats

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  e) Activation Stats

In [9]:
ActivationStats(PilotNet,test_set,15,0,107)

mean value (MMU): -0.015098267
mean value (Buffer): 0.054420285
maximum (MMU): 1.3855873
minimum (MMU): -1.5350285
maximum (Buffer): 1.3855873
minimum (Buffer): -1.0
saturation ratio (MMU): 2.8940604173014062e-05
saturation ratio (Buffer): 0.0011816785050555553


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  f) Write/Read Stats

In [19]:
Indices = [5,6,10,14,18,22,28,32,36,40,44]
#Data    = GetReadAndWrites(PilotNet,Indices,72912,150,CNN_gating=False)
#stats   = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
#Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
Data   = GetReadAndWrites(PilotNet,Indices,72912,150,CNN_gating=True)
stats    = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
CNN_gating_Acceses = pd.DataFrame(stats).reset_index(drop=False)
#save_obj(Baseline_Acceses,'Data/Acceses/PilotNet/Baseline')
#save_obj(CNN_gating_Acceses,'Data/Acceses/PilotNet/CNN_gating_Adj')

procesados:  25
procesados:  50
procesados:  75
procesados:  100
procesados:  125
procesados:  150


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Starting Point

In [24]:
CheckAccuracyAndLoss('PilotNet', test_dataset, wgt_dir, act_frac_size = 15, act_int_size = 0, wgt_frac_size = 15, wgt_int_size = 0, 
                    input_shape = (160,320,3), output_shape = 1, batch_size = testBatchSize);

dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
dentro de GenerateAddressList 
estoy en WeightQuantization
107/107 [==============================] - 7s 54ms/step - loss: 0.0024


### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b) Number of bits analysis

# 4) Error Injection

In [25]:
Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216


ruta_bin = 'Data/Fault Characterization/VC707/RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)


ficheros = [fichero.name for fichero in directorio.iterdir()]
ficheros.sort()

for i, j in enumerate(ficheros):
    directorio= os.path.join(ruta_bin, j)
    buffer= (analize_file(directorio, buffer_size))
    error_mask, locs = (buffer_vectores(buffer))
    print(directorio)

    loss,acc   = CheckAccuracyAndLoss('PilotNet', test_dataset, wgt_dir, output_shape=8,  input_shape = (160,320,3),
                                            act_frac_size = 15, act_int_size = 0, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)

   
    loss,acc   = CheckAccuracyAndLoss('PilotNet', test_dataset, wgt_dir, output_shape=8,  input_shape = (160,320,3),
                                            act_frac_size = 15, act_int_size = 0, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
   
    loss,acc   = CheckAccuracyAndLoss('PilotNet', test_dataset, wgt_dir, output_shape=8,  input_shape = (160,320,3),
                                            act_frac_size = 15, act_int_size = 0, wgt_frac_size = 15, wgt_int_size = 0,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)


    Accs.append(acc)
    Accs_w.append(acc_w)
    Accs_a_w.append(acc_a_w)


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_a_w)
#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero']
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)


Acc.head()

soy el buffer dentro de funciones 16777216
 dentro de funciones Cantidad de elementos por tipo : Counter({'x': 16766856, '0': 10360})
Data/Fault Characterization/VC707/RawData\VC707-0.54.bin
dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro d

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
d

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

...

dentro de GenerateAddressList 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
d

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de DecodeMask 
dentro de 

ValueError: Shapes (8,) and (1,) are incompatible